### Preparations

To get started let's include some packages and modules that will be useful.

In [10]:
from bloqade import rydberg_h, piecewise_linear, piecewise_constant, waveform, cast
from bloqade.atom_arrangement import ListOfLocations, Lieb, Square, Chain, Honeycomb, Kagome, Triangular, Rectangular
from bokeh.io import output_notebook 
import os
from math import pi
import matplotlib.pyplot as plt
import numpy as np

output_notebook()
from bloqade import var, save, load
from bloqade.atom_arrangement import *
import matplotlib.pyplot as plt
import numpy as np
import os

if not os.path.isdir("data"):
    os.mkdir("data")

t = np.linspace(0, 2 * pi, 7)
values = list(zip(2 + 1 * np.cos(t), 1.5 + .5 * np.sin(t)))
run_time = var("run_time")

quantum_scar_program = (
    ListOfLocations(values)
   
    .rydberg.detuning.uniform.piecewise_linear(
        [0.3, 1.6, 0.3], [-45, -45, 39, 39]
    )
    .piecewise_linear([0.2, 1.6], [39, 0.0, 0.0])
    .slice(start=0, stop=run_time)
    .amplitude.uniform.piecewise_linear([0.3, 1.6, 0.3], [0.0, 15.7, 15.7, 0.0])
    .piecewise_linear([0.2, 1.4, 0.2], [0, 15.7, 15.7, 0])
    .slice(start=0, stop=run_time - 0.065)
    .record("rabi_value")
    
    .linear("rabi_value", 0, 0.35)
)

prep_times = np.arange(0.2, 2.2, 0.2)
scar_times = np.arange(2.2, 4.01, 0.01)
run_times = np.unique(np.hstack((prep_times, scar_times)))

batch = quantum_scar_program.batch_assign(run_time=run_times)

Loading BokehJS ...

In [11]:
emu_filename = os.path.join(
    os.path.abspath(""), "data", "quantum-scar-dynamics-emulation6.json"
)

emu_batch = batch.bloqade.python().run(shots=100, interaction_picture = True)
save(emu_batch, emu_filename)

In [ ]:
def get_z2_probabilities(report):
    z2_probabilities = []

    for count in report.counts():
        z2_probability = count.get("101010101010", 0) / sum(list(count.values()))
        z2_probabilities.append(z2_probability)

    return z2_probabilities

In [ ]:
emu_report = emu_batch.report()
#hardware_report = hardware_batch.report()

emu_run_times = emu_report.list_param("run_time")
emu_z2_prob = get_z2_probabilities(emu_report)

#hw_run_times = hardware_report.list_param("run_time")
#hw_z2_prob = get_z2_probabilities(hardware_report)

plt.plot(emu_run_times, emu_z2_prob, label="Emulator", color="#878787")
#plt.plot(hw_run_times, hw_z2_prob, label="QPU", color="#6437FF")

plt.legend()
plt.xlabel("Time ($\mu s$)")
plt.ylabel("Z2-state Probability")
plt.show()


In [7]:
#geometry
rng = np.random.default_rng(1234)
# atom_pos=Square(5, lattice_spacing=4.5).apply_defect_density(0.3, rng=rng).remove_vacant_sites()
t = np.linspace(0, 2 * pi, 12)
values = list(zip(2 + 1 * np.cos(t), 1.5 + .5 * np.sin(t)))

#dynamics
durations = [0.15,3.7,0.15]
delta_MHz=[-13.0,-13.0,11.0,11.0]
omega_MHz= [0.0,2.5,2.5,0.0]

Delta = piecewise_linear(durations,[x*2*np.pi for x in delta_MHz])
Omega = piecewise_linear(durations,[x*2*np.pi for x in omega_MHz])

#create Hamiltonian
program = rydberg_h(values, detuning= Delta, amplitude=Omega, phase=None)

program.parse_register().show()
program.parse_sequence().show()

Given a Hamiltonian describing a full program, we might as well run it to see what it does! Let's try first emulating the calculation.